In [25]:
import numpy as np
import os
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm

from collections import Counter
import matplotlib.pyplot as plt
# !unzip "cora (extract.me).zip"


In [26]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

In [27]:
categories =  ['Reinforcement_Learning', 'Theory', 'Case_Based', 'Genetic_Algorithms', 'Probabilistic_Methods', 'Neural_Networks', 'Rule_Learning']
sorted(categories)
label_encoder = {}
i = 0
for cat in sorted(categories):
  label_encoder[cat] = i
  i +=1
label_encoder

{'Case_Based': 0,
 'Genetic_Algorithms': 1,
 'Neural_Networks': 2,
 'Probabilistic_Methods': 3,
 'Reinforcement_Learning': 4,
 'Rule_Learning': 5,
 'Theory': 6}

In [28]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features
print('X shape: ', X.shape)

#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

print('\nNumber of nodes (N): ', N)
print('\nNumber of features (F) of each node: ', F)
print('\nCategories: ', set(labels))

num_classes = len(set(labels))
print('\nNumber of classes: ', num_classes)

X shape:  (2708, 1433)

Number of nodes (N):  2708

Number of features (F) of each node:  1433

Categories:  {0, 1, 2, 3, 4, 5, 6}

Number of classes:  7


In [29]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
print('Graph info: ', nx.info(G))

Graph info:  Graph with 2708 nodes and 5278 edges


In [31]:
nodes = list(G.nodes)
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
df.head()

2708


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)


In [33]:
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes 
print(len(df))
df.head()

2485


,node,label,features
0,0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [34]:
train = df.groupby('label', group_keys=False).apply(lambda x: x.sample(20))
G = nx.relabel_nodes(G, df['node'])

In [35]:
def create_transition_matrix(g):
    vs = list(g.nodes)
    n = len(vs)
    adj = nx.adjacency_matrix(g)
    transition_matrix = adj/adj.sum(axis=1)

    return transition_matrix

In [36]:
def random_walk(g, num_steps, start_node, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
    print( 'create_transition_matrix')
  #perform a random walk 
  v = start_node
  all_node = np.arange(len(transition_matrix))
  for i in range(num_steps):
    connected = (transition_matrix[v]!=0)
    p = transition_matrix[v][connected]
    node = all_node[connected]
    v = np.random.choice(node, p = p)

  return v

In [37]:
seeds_dict = {predicted:list(train[train['label'] == predicted]['node']) for predicted in range(7)}

def random_walk_with_teleportation(g, num_steps, start_node, tp, predicted = None, transition_matrix = None):
  if transition_matrix is None:
    transition_matrix = create_transition_matrix(g)
  v = start_node
  all_node = np.arange(len(transition_matrix))
  for i in range(num_steps):
    if(np.random.uniform() < tp):
      v = np.random.choice(predicted)
    else:
      connected = (transition_matrix[v]!=0)
      p= transition_matrix[v][connected]
      node = all_node[connected]
      v = np.random.choice(node, p = p)
    
  return v

In [38]:
# Part A. W.o. teleportation, w.o. tfidf. 
transition_matrix = create_transition_matrix(G)
transition_matrix = np.array(transition_matrix)
num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

for train_node, predicted in zip(train['node'],train['label']):
  for i in range(num_samples):
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
print('accuracy_score', accuracy_score(df['label'], preds))
print('f1_score', f1_score(df['label'], preds, average='macro'))

unvisited =  0
              precision    recall  f1-score   support

           0       0.36      0.57      0.44       285
           1       0.46      0.51      0.48       406
           2       0.53      0.30      0.38       726
           3       0.65      0.59      0.62       379
           4       0.12      0.16      0.14       214
           5       0.10      0.21      0.13       131
           6       0.21      0.17      0.19       344

    accuracy                           0.37      2485
   macro avg       0.35      0.36      0.34      2485
weighted avg       0.41      0.37      0.38      2485

accuracy_score 0.3734406438631791
f1_score 0.34015956840581435


In [39]:
# Part B. W. teleportation, w.o. tfidf. 
transition_matrix = create_transition_matrix(G)
transition_matrix = np.array(transition_matrix)
num_samples = 1000  
num_walk_steps = 100

for tp in [0,0.1,0.2]:
  visiting_freq_label = []
  for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])
  visiting_freq = [0 for i in range(transition_matrix.shape[0])]

  for train_node, predicted in zip(train['node'],train['label']):
    for i in range(num_samples):
        start_point = train_node
        end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted = seeds_dict[predicted], transition_matrix = transition_matrix)
        visiting_freq_label[end_node][predicted] += 1
        visiting_freq[end_node] +=1
  
  count = 0 #these many nodes remain unvisited. 
  for vf in visiting_freq:
    if vf ==0:
      count+=1
  print('unvisited = ', count)
  visiting_freq_label = np.asarray(visiting_freq_label)
  preds = np.argmax(visiting_freq_label,axis = 1)
  print(classification_report(df['label'], preds))
  print('accuracy_score', accuracy_score(df['label'], preds))
  print('f1_score', f1_score(df['label'], preds, average='macro'))

unvisited =  0
              precision    recall  f1-score   support

           0       0.37      0.61      0.46       285
           1       0.47      0.51      0.49       406
           2       0.55      0.28      0.37       726
           3       0.68      0.56      0.62       379
           4       0.09      0.14      0.11       214
           5       0.10      0.24      0.15       131
           6       0.26      0.20      0.22       344

    accuracy                           0.38      2485
   macro avg       0.36      0.36      0.35      2485
weighted avg       0.43      0.38      0.39      2485

accuracy_score 0.3750503018108652
f1_score 0.346538662493233
unvisited =  23
              precision    recall  f1-score   support

           0       0.71      0.72      0.72       285
           1       0.83      0.89      0.86       406
           2       0.86      0.56      0.67       726
           3       0.75      0.79      0.77       379
           4       0.59      0.83      0

In [40]:
# compute similarity for transition matrix
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_feat = np.array(TfidfTransformer().fit_transform(np.stack(df.features)).todense())
from math import *
vs = list(G.nodes)
n = len(vs)
adj = nx.adjacency_matrix(G)
transition = np.zeros((len(G.nodes), len(G.nodes)))

transition_matrix = np.array(create_transition_matrix(G))

for n1 in df['node'].tolist():
  for n2 in df['node'].tolist():
    if transition_matrix[n1][n2]:
      assert transition_matrix[n2][n1]>0
      a, b = tfidf_feat[n1], tfidf_feat[n2]
      cos_sim = (a @ b.T) / (norm(a)*norm(b))
      transition[n1,n2] = np.exp(cos_sim) 

for i in range(len(transition)):
  line = transition[i][transition[i]!=0]
  transition[i] /= line.sum()

In [41]:
# Part C. W.o. teleportation, w. tfidf. 
transition_matrix = transition
num_samples = 1000  
num_walk_steps = 100

visiting_freq_label = []
for i in range(transition_matrix.shape[0]):
  visiting_freq_label.append([0,0,0,0,0,0,0])

visiting_freq = [0 for i in range(transition_matrix.shape[0])]

for train_node,predicted in tqdm(zip(train['node'],train['label']), total=len(train['label'])):
  for i in range(num_samples):
      start_point = train_node
      end_node = random_walk(G, num_walk_steps, start_point, transition_matrix)
      visiting_freq_label[end_node][predicted] += 1
      visiting_freq[end_node] +=1

count = 0 #these many nodes remain unvisited. 
for vf in visiting_freq:
  if vf ==0:
    count+=1
print('unvisited = ', count)
visiting_freq_label = np.asarray(visiting_freq_label)
preds = np.argmax(visiting_freq_label,axis = 1)
print(classification_report(df['label'], preds))
print('accuracy_score', accuracy_score(df['label'], preds))
print('f1_score', f1_score(df['label'], preds, average='macro'))

100%|██████████| 140/140 [07:53<00:00,  3.38s/it]

unvisited =  0
              precision    recall  f1-score   support

           0       0.36      0.58      0.44       285
           1       0.45      0.53      0.49       406
           2       0.55      0.28      0.37       726
           3       0.69      0.58      0.63       379
           4       0.12      0.17      0.14       214
           5       0.11      0.26      0.16       131
           6       0.30      0.22      0.25       344

    accuracy                           0.38      2485
   macro avg       0.37      0.38      0.36      2485
weighted avg       0.44      0.38      0.39      2485

accuracy_score 0.3835010060362173
f1_score 0.3561698600914448


In [42]:
# Part D. W. teleportation, w. tfidf. 
transition_matrix = transition
num_samples = 1000  
num_walk_steps = 100

for tp in [0,0.1,0.2]:
  visiting_freq_label = []
  for i in range(transition_matrix.shape[0]):
    visiting_freq_label.append([0,0,0,0,0,0,0])
  visiting_freq = [0 for i in range(transition_matrix.shape[0])]

  for train_node, predicted in tqdm(zip(train['node'],train['label']), total=len(train['label'])):
    for i in range(num_samples):
        start_point = train_node
        end_node = random_walk_with_teleportation(G, num_walk_steps, start_point, tp, predicted = seeds_dict[predicted], transition_matrix = transition_matrix)
        visiting_freq_label[end_node][predicted] += 1
        visiting_freq[end_node] +=1
  
  count = 0 #these many nodes remain unvisited. 
  for vf in visiting_freq:
    if vf ==0:
      count+=1
  print('unvisited = ', count)
  visiting_freq_label = np.asarray(visiting_freq_label)
  preds = np.argmax(visiting_freq_label,axis = 1)
  print(classification_report(df['label'], preds))
  print('accuracy_score', accuracy_score(df['label'], preds))
  print('f1_score', f1_score(df['label'], preds, average='macro'))

100%|██████████| 140/140 [08:46<00:00,  3.76s/it]


unvisited =  1
              precision    recall  f1-score   support

           0       0.36      0.59      0.45       285
           1       0.52      0.58      0.55       406
           2       0.59      0.31      0.40       726
           3       0.64      0.59      0.62       379
           4       0.14      0.21      0.17       214
           5       0.11      0.26      0.16       131
           6       0.27      0.20      0.23       344

    accuracy                           0.40      2485
   macro avg       0.38      0.39      0.37      2485
weighted avg       0.45      0.40      0.41      2485

accuracy_score 0.4
f1_score 0.36767830318577926


100%|██████████| 140/140 [08:50<00:00,  3.79s/it]


unvisited =  27
              precision    recall  f1-score   support

           0       0.70      0.71      0.71       285
           1       0.84      0.90      0.87       406
           2       0.87      0.53      0.66       726
           3       0.75      0.78      0.76       379
           4       0.54      0.82      0.65       214
           5       0.43      0.84      0.57       131
           6       0.62      0.63      0.63       344

    accuracy                           0.71      2485
   macro avg       0.68      0.74      0.69      2485
weighted avg       0.74      0.71      0.71      2485

accuracy_score 0.7054325955734406
f1_score 0.6937672972410642


100%|██████████| 140/140 [08:33<00:00,  3.67s/it]

unvisited =  89
              precision    recall  f1-score   support

           0       0.58      0.71      0.64       285
           1       0.85      0.88      0.87       406
           2       0.86      0.54      0.66       726
           3       0.74      0.76      0.75       379
           4       0.58      0.81      0.68       214
           5       0.44      0.83      0.58       131
           6       0.60      0.58      0.59       344

    accuracy                           0.69      2485
   macro avg       0.67      0.73      0.68      2485
weighted avg       0.73      0.69      0.69      2485

accuracy_score 0.6921529175050302
f1_score 0.6804685650782573
